
# Assignment 3

This is a template notebook for Assignment 3.


## Install dependencies and initialization

In [ ]:
# The next 8 lines added to be compatible with version changes (2022/10/13).
# change cuda to 11.1
import os
p = os.getenv('PATH')
ld = os.getenv('LD_LIBRARY_PATH')
os.environ['PATH'] = f"/usr/local/cuda-11.1/bin:{p}"
os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-11.1/lib64:{ld}"
# change pytorch to 1.9.0 compiled with cuda 11.1
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html


# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html

In [ ]:
!pwd # shows current directory
!ls  # shows all files in this directory
!nvidia-smi # shows the specs and the current status of the allocated GPU

In [ ]:
# import some common libraries
from google.colab.patches import cv2_imshow
from sklearn.metrics import jaccard_score
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import datetime
import random
import json
import cv2
import csv
import os

# import some common pytorch utilities
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch

# import some common detectron2 utilities
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
setup_logger()

In [ ]:
# Make sure that GPU is available for your notebook. 
# Otherwise, you need to update the settungs in Runtime -> Change runtime type -> Hardware accelerator
import torch
torch.cuda.is_available()

In [ ]:
# You need to mount your google drive in order to load the data:
from google.colab import drive
drive.mount('/content/drive')
# Put all the corresponding data files in a data folder and put the data folder in a same directory with this notebook.
# Also create an output directory for your files such as the trained models and the output images.

In [ ]:
# Define the location of current directory, which should contain data/train, data/test, and data/train.json.
# TODO: approx 1 line
BASE_DIR = '/content/drive/MyDrive/CMPT_CV_lab3/data'
OUTPUT_DIR = '{}/output'.format(BASE_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Part 1: Object Detection

### Data Loader

In [ ]:
from zmq.constants import NULL
from os import listdir
from os.path import isfile, join
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
'''
# This function should return a list of data samples in which each sample is a dictionary. 
# Make sure to select the correct bbox_mode for the data
# For the test data, you only have access to the images, therefore, the annotations should be empty.
# Other values could be obtained from the image files.
# TODO: approx 35 lines
'''

def get_detection_data(set_name):

  # Save result in pkl file to speed up repeated access
  # import pickle

  # if set_name == "test":
  #   with open('/content/drive/MyDrive/CMPT_CV_lab3/data/dataset_dicts_test.pkl', 'rb') as f:
  #       result = pickle.load(f)

  # if set_name == "train":
  #   with open('/content/drive/MyDrive/CMPT_CV_lab3/data/dataset_dicts_train.pkl', 'rb') as f:
  #       result1 = pickle.load(f)

  #   with open('/content/drive/MyDrive/CMPT_CV_lab3/data/dataset_dicts_val.pkl', 'rb') as f:
  #       result2 = pickle.load(f)

  #   result = result1 + result2


  if set_name == "train":
    json_file = os.path.join(BASE_DIR, "train", "train.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)
    dataset = {}

    for v in imgs_anns:
        if v["image_id"] not in dataset:
          record = {}
          file_name = os.path.join(BASE_DIR, "train", v["file_name"])
          img = cv2.imread(file_name)
          if img is None:
            img = Image.open(file_name)
            img = np.asarray(img)
          if img is not None:
            height, width = img.shape[:2]
            record["file_name"] = file_name
            record["image_id"] = v["image_id"]
            record["height"] = height
            record["width"] = width
            record["annotations"] = []

            obj = {
                "bbox": v["bbox"],
                "bbox_mode": BoxMode.XYWH_ABS,
                "segmentation": v["segmentation"],
                "category_id": 0,
                "iscrowd" : v["iscrowd"]
            }
            record["annotations"].append(obj)
            dataset[v["image_id"]] = record
        else:
          obj = {
              "bbox": v["bbox"],
              "bbox_mode": BoxMode.XYWH_ABS,
              "segmentation": v["segmentation"],
              "category_id": 0,
              "iscrowd" : v["iscrowd"]
          }
          dataset[v["image_id"]]["annotations"].append(obj)
  elif set_name == "test":
    image_id = 0
    dataset = {}

    for f in listdir(join(BASE_DIR, set_name)):
      if isfile(join(BASE_DIR, set_name, f)):
        record = {}
        file_name = join(BASE_DIR, set_name, f)
        img = cv2.imread(file_name)
        if img is None:
          img = Image.open(file_name)
          img = np.asarray(img)
        if img is not None:
          height, width = img.shape[:2]
          record["file_name"] = file_name
          record["image_id"] = image_id
          record["height"] = height
          record["width"] = width
          record["annotations"] = []
          dataset[f] = record
      image_id += 1

  result = list(dataset.values())

  
  return result

In [ ]:
'''
# Remember to add your dataset to DatasetCatalog and MetadataCatalog
# Consdier "data_detection_train" and "data_detection_test" for registration
# You can also add an optional "data_detection_val" for your validation by spliting the training data
# TODO: approx 5 lines
'''

for d in ["train", "test"]:
    DatasetCatalog.register("data_detection_" + d, lambda d=d: get_detection_data(d))
    MetadataCatalog.get("data_detection_" + d).set(thing_classes=["planes"])
data_detection_train_metadata = MetadataCatalog.get("data_detection_train")
data_detection_test_metadata = MetadataCatalog.get("data_detection_test")

In [ ]:
'''
# Visualize some samples using Visualizer to make sure that the function works correctly
# TODO: approx 5 lines
'''
dataset_dicts_train = get_detection_data("train")
dataset_dicts_test = get_detection_data("test")
for d in random.sample(dataset_dicts_train, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=data_detection_train_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])


In [ ]:
import pickle
# with open('/content/drive/MyDrive/CMPT_CV_lab3/data/dataset_dicts_test.pkl', 'wb') as f:
#     pickle.dump(dataset_dicts_test, f)

# with open('/content/drive/MyDrive/CMPT_CV_lab3/data/dataset_dicts_train.pkl', 'wb') as f:
#     pickle.dump(dataset_dicts_train, f)

# with open('/content/drive/MyDrive/CMPT_CV_lab3/data/dataset_dicts_val.pkl', 'wb') as f:
#     pickle.dump(dataset_dicts_val, f)

### Set Configs

In [ ]:
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_detection_train",)
cfg.DATASETS.TEST = ("data_detection_test",)
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.003 
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
cfg.SOLVER.STEPS = []
cfg.MODEL.PIXEL_STD = [57.375, 57.120, 58.395]
cfg.SOLVER.MOMENTUM = 0.9
cfg.SOLVER.MAX_ITER = 2000  
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512  
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 

### Training

In [ ]:
'''
# Create a DefaultTrainer using the above config and train the model
# TODO: approx 5 lines
'''

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

### Evaluation and Visualization

In [ ]:
'''
# After training the model, you need to update cfg.MODEL.WEIGHTS
# Define a DefaultPredictor
'''
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6 
cfg.DATASETS.TEST = ("data_detection_train",)
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.data.build import get_detection_dataset_dicts
'''
# Visualize the output for 3 random test samples
# TODO: approx 10 lines
'''
from detectron2.utils.visualizer import ColorMode
test_dataset_dicts = get_detection_data("test")
for d in random.sample(test_dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    # im = torch.tensor(im, device=torch.device('cuda'), dtype=torch.float)
    outputs = predictor(im)  
    v = Visualizer(im[:, :, ::-1],
                   metadata=data_detection_test_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW 
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
'''
# Use COCOEvaluator and build_detection_train_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("data_detection_train", output_dir="/content/drive/MyDrive/CMPT_CV_lab3/data/output/")
test_loader = build_detection_test_loader(cfg, "data_detection_train")
print(inference_on_dataset(predictor.model, test_loader, evaluator))


In [ ]:
%load_ext tensorboard
%tensorboard --logdir "/content/drive/MyDrive/CMPT_CV_lab3/data/output/"

### Improvements

For this part, you can bring any improvement which you have by adding new input parameters to the previous functions or defining new functions and variables.

In [ ]:
'''
# Bring any changes and updates regarding the improvement in here
'''


## Part 2: Semantic Segmentation

### Data Loader

In [ ]:
from re import X
'''
# Write a function that returns the cropped image and corresponding mask regarding the target bounding box
# idx is the index of the target bbox in the data
# high-resolution image could be passed or could be load from data['file_name']
# You can use the mask attribute of detectron2.utils.visualizer.GenericMask 
#     to convert the segmentation annotations to binary masks
# TODO: approx 10 lines
'''

def get_instance_sample(data, idx, img=None):
    d = list(data.values())[4]
    x =  int(d[idx]["bbox"][0])
    y =  int(d[idx]["bbox"][1])
    w =  int(d[idx]["bbox"][2])
    h =  int(d[idx]["bbox"][3])
    obj_img = img[y:y+h, x:x+w, :]
    obj_mask = np.zeros_like((obj_img.shape[0], obj_img.shape[1]))
    if "segmentation" in d[idx]:
      obj = detectron2.utils.visualizer.GenericMask(d[idx]["segmentation"], data["height"], data["width"])
      obj_mask = obj.mask
      # plt.figure()
      # plt.imshow(obj_mask, cmap='gray')
      obj_mask = obj_mask[y:y+h, x:x+w]
      # plt.figure()
      # plt.imshow(obj_mask, cmap='gray')
    return obj_img, obj_mask

In [ ]:
'''
# We have provided a template data loader for your segmentation training
# You need to complete the __getitem__() function before running the code
# You may also need to add data augmentation or normalization in here
'''

class PlaneDataset(Dataset):
  def __init__(self, set_name, data_list):
      self.transforms = transforms.Compose([
          # transforms.ToTensor(),
          # transforms.Resize((256,256)),
          # transforms.RandomCrop((128,128), padding=4, padding_mode="reflect"), 
          # transforms.RandomRotation(15),
          # transforms.RandomHorizontalFlip(),
          # transforms.Normalize()
      ])
      self.set_name = set_name
      self.data = data_list
      self.instance_map = []
      for i, d in enumerate(self.data):
        for j in range(len(d['annotations'])):
          self.instance_map.append([i,j])

  '''
  # you can change the value of length to a small number like 10 for debugging of your training procedure and overfeating
  # make sure to use the correct length for the final training
  '''
  def __len__(self):
      return len(self.instance_map)

  def numpy_to_tensor(self, img, mask):
    if self.transforms is not None:
        img = self.transforms(img)
    img = torch.tensor(img, dtype=torch.float)
    mask = torch.tensor(mask, dtype=torch.float)
    return img, mask

  '''
  # Complete this part by using get_instance_sample function
  # make sure to resize the img and mask to a fixed size (for example 128*128)
  # you can use "interpolate" function of pytorch or "numpy.resize"
  # TODO: 5 lines
  '''
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
        idx = idx.tolist()
    idx = self.instance_map[idx]
    data = self.data[idx[0]]
    idx = idx[1]
    file_name = data["file_name"]
    img = cv2.imread(file_name)
    if img is None:
      img = Image.open(file_name)
      img = np.asarray(img)
    img, mask = get_instance_sample(data, idx, img)
    img, mask = self.numpy_to_tensor(img, mask)
    img = torch.permute(img, (2, 0, 1))
    img = torch.nn.functional.interpolate(img.unsqueeze(0).float(), size=(256,256), mode='nearest').squeeze(0)
    mask = mask.unsqueeze(0)
    mask = torch.nn.functional.interpolate(mask.unsqueeze(0).float(), size=(256,256), mode='nearest').squeeze(0)
    return img, mask

def get_plane_dataset(set_name='train', batch_size=2):
    my_data_list = DatasetCatalog.get("data_detection_{}".format(set_name))
    dataset = PlaneDataset(set_name, my_data_list)
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=4,
                                              pin_memory=True, shuffle=True)
    return loader, dataset

### Network

In [ ]:
'''
# convolution module as a template layer consists of conv2d layer, batch normalization, and relu activation
'''
class conv(nn.Module):
    def __init__(self, in_ch, out_ch, activation=True):
        super(conv, self).__init__()
        if(activation):
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1),
             nn.BatchNorm2d(out_ch),
             nn.ReLU(inplace=False)
          )
        else:
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1)  
             )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# downsampling module equal to a conv module followed by a max-pool layer
'''
class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.layer = nn.Sequential(
            conv(in_ch, out_ch, True),
            nn.MaxPool2d(2)
            )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# upsampling module equal to a upsample function followed by a conv module
'''
class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super(up, self).__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch, in_ch, 2, stride=2)

        self.conv = conv(in_ch, out_ch, True)

    def forward(self, x):
        y = self.up(x)
        y = self.conv(y)
        return y

'''
# the main model which you need to complete by using above modules.
# you can also modify the above modules in order to improve your results.
'''
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        
        # Encoder
        
        self.input_conv = conv(3, 16, True)
        self.down1 = down(16, 32)
        self.conv2 = conv(32, 32, True)
        self.down2 = down(32, 64)
        self.down3 = down(64, 128)
        self.conv3 = conv(128, 128, True)
        self.conv4 = conv(128, 512, True)
        
        # Decoder
        
        self.conv5 = conv(512, 128, True)
        self.conv6 = conv(128, 128, True)
        self.up1 = up(128, 64)
        self.up2 = up(64, 32)
        self.conv7 = conv(32, 32, True)
        self.up3 = up(32, 16)
        self.conv8 = conv(16, 3, True)
        self.output_conv = conv(3, 1, False) # ReLu activation is removed to keep the logits for the loss function
        

    def forward(self, input):
      y = self.input_conv(input)
      y = self.down1(y)
      y1 = self.conv2(y)
      y1 += y
      y = self.down2(y1)
      y = self.down3(y)
      y1 = self.conv3(y)
      y1 += y
      y = self.conv4(y1)
      y = self.conv5(y)
      y1 = self.conv6(y)
      y1 += y
      y = self.up1(y1)
      y = self.up2(y)
      y1 = self.conv7(y)
      y1 += y
      y = self.up3(y1)
      y = self.conv8(y)
      output = self.output_conv(y)
      return output

### Training

In [ ]:
'''
# The following is a basic training procedure to train the network
# You need to update the code to get the best performance
# TODO: approx ? lines
'''
# Set the hyperparameters
num_epochs = 10
batch_size = 4
learning_rate = 0.008
weight_decay = 3e-5

model = MyModel() # initialize the model
model = model.cuda() # move the model to GPU

loader, _ = get_plane_dataset('train', batch_size) # initialize data_loader
crit = nn.BCEWithLogitsLoss() # Define the loss function
optim = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
sch = torch.optim.lr_scheduler.CosineAnnealingLR(optim, T_max=num_epochs)
# ix = 0
# start the training procedure
for epoch in range(3,num_epochs):
  total_loss = 0
  for (img, mask) in tqdm(loader):
    img = img.cuda()
    mask = mask.cuda()
    pred = model(img)
    loss = crit(pred, mask)
    optim.zero_grad()
    loss.backward()
    optim.step()
    total_loss += loss.cpu().data
    # ix += 1
    # if ix % 100 == 0:
    #   print(total_loss)
  print("Epoch: {}, Loss: {}".format(epoch, total_loss/len(loader)))
  torch.save(model.state_dict(), '{}/output/{}_segmentation_model.pth'.format(BASE_DIR, epoch))
  sch.step()

'''
# Saving the final model 
'''
torch.save(model.state_dict(), '{}/output/final_segmentation_model.pth'.format(BASE_DIR))


In [ ]:
# Taken from the training runs
Epoch = [0,1,2,3,4,5,6,7,8,9]
Loss = [0.29557718515396, 0.250679028034, 0.235619907896, 0.21746918559074402, 0.2146557718515396, 0.21142378449440002, 0.2089690864086151, 0.20506790280342102, 0.20036287605762482,  0.19746056199073792
]

plt.plot(Epoch, Loss) 
plt.grid()  
plt.xlabel('Epochs') 
plt.ylabel('Training Loss') 
plt.title('Loss vs Epochs') 
plt.show() 

### Evaluation and Visualization

In [ ]:
from IPython.core.profiledir import ProfileDir
'''
# Before starting the evaluation, you need to set the model mode to eval
# You may load the trained model again, in case if you want to continue your code later
# TODO: approx 15 lines
'''
batch_size = 4
model = MyModel().cuda()
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))
model = model.eval() # chaning the model to evaluation mode will fix the bachnorm layers
loader, dataset = get_plane_dataset('train', batch_size)

total_iou = 0
no_of_batches = 0
for (img, mask) in tqdm(loader):
  with torch.no_grad():
    no_of_batches += 1
    img = img.cuda()
    mask = mask.cuda()
    pred = model(img)
    threshold = (torch.min(pred) + torch.max(pred)) / 2
    pred[pred >= threshold] = 1
    pred[pred < threshold] = 0
    pred = pred.cpu().numpy()
    mask = mask.cpu().numpy()
    intersection = np.logical_and(mask, pred)
    union = np.logical_or(mask, pred)
    total_iou += np.sum(intersection) / np.sum(union)
    '''
    ## Complete the code by obtaining the IoU for each img and print the final Mean IoU
    '''
    
 
print("\n #images: {}, Mean IoU: {}".format(no_of_batches * batch_size, total_iou / no_of_batches))


In [ ]:
'''
# Visualize 3 sample outputs
# TODO: approx 5 lines
'''


from matplotlib import pyplot as plt

from detectron2.utils.visualizer import ColorMode
test_dataset_dicts = get_detection_data("test")
for d in random.sample(test_dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])

    outputs = predictor(im) 
    box_iter = iter(outputs["instances"].get_fields()["pred_boxes"])
    try:
      item = next(box_iter)
      item = item.cpu().numpy()
      x =  round(item[0])
      y =  round(item[1])
      w =  round(item[2] - item[0])
      h =  round(item[3] - item[1])
      img1 = im[y:y+h, x:x+w, :]
    except Exception as e:
      continue

    img = torch.tensor(img1, dtype=torch.float)
    img = torch.permute(img, (2, 0, 1))
    img = torch.nn.functional.interpolate(img.unsqueeze(0).float(), size=(256,256), mode='nearest').squeeze(0)
    img = img.unsqueeze(0)
    img = img.cuda()
    with torch.no_grad():
      pred = model(img)  
    pred = pred.squeeze(0).squeeze(0)
    threshold = (torch.min(pred) + torch.max(pred)) / 2
    pred[pred >= threshold] = 1
    pred[pred < threshold] = 0
    
    pred = pred.detach().cpu().numpy()
    plt.figure()
    plt.imshow(pred, cmap='gray')
    plt.figure()
    plt.imshow(img1)


    # img, mask = get_instance_sample(d, 0, im)



## Part 3: Instance Segmentation

In this part, you need to obtain the instance segmentation results for the test data by using the trained segmentation model in the previous part and the detection model in Part 1.

### Get Prediction

In [ ]:
from matplotlib.pyplot import xticks
'''
# Define a new function to obtain the prediction mask by passing a sample data
# For this part, you need to use all the previous parts (predictor, get_instance_sample, data preprocessings, etc)
# It is better to keep everything (as well as the output of this funcion) on gpu as tensors to speed up the operations.
# pred_mask is the instance segmentation result and should have different values for different planes.
# TODO: approx 35 lines
'''

count = 0 
model = MyModel().cuda()
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))
model = model.eval() # chaning the model to evaluation mode will fix the bachnorm layers

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6 
predictor = DefaultPredictor(cfg)

def get_prediction_mask(data):
    gt_masks_list = []
    pred_masks_list = []
    file_name = data['image_id']
    img = cv2.imread(file_name)
    if img is None:
      img = Image.open(file_name)
      img = np.asarray(img)
    gt_mask = np.zeros([img.shape[0], img.shape[1]])
    pred_mask = np.zeros([img.shape[0], img.shape[1]])
    
    if len(data["annotations"]) > 0:
      for i in range(len(data["annotations"])):
        im, mask = get_instance_sample(data, i, img)
        d = list(data.values())[4]
        x =  int(d[i]["bbox"][0])
        y =  int(d[i]["bbox"][1])
        w =  int(d[i]["bbox"][2])
        h =  int(d[i]["bbox"][3])
        
        gt_mask[y:y+h, x:x+w] = mask * (i + 1)
        mask = torch.tensor(mask, dtype=torch.int32)
        gt_masks_list.append(mask)
    
    with torch.no_grad():
      outputs = predictor(img) 
    box_iter = iter(outputs["instances"].get_fields()["pred_boxes"])
    counter = 0
    while True:
      try:
        item = next(box_iter)
        counter += 1
        item = item.cpu().numpy()
        x =  round(item[0])
        y =  round(item[1])
        w =  round(item[2] - item[0])
        h =  round(item[3] - item[1])
        obj = {}
        obj["bbox"] = [x,y,w,h]
        data["annotations"] = [obj]
        im, mask = get_instance_sample(data, 0, img)
        im = torch.tensor(im, dtype=torch.float)
        im = torch.permute(im, (2, 0, 1))
        im = torch.nn.functional.interpolate(im.unsqueeze(0).float(), size=(256,256), mode='nearest').squeeze(0)
        im = im.unsqueeze(0)
        im = im.cuda()
        with torch.no_grad():
          pred = model(im)
        pred = pred.squeeze(0).squeeze(0)
        threshold = (torch.min(pred) + torch.max(pred)) / 2
        pred[pred >= threshold] = 1
        pred[pred < threshold] = 0
        pred = pred.detach().cpu().numpy()
        # plt.figure()
        # plt.imshow(pred, cmap='gray')
        s = pred_mask[y:y+h, x:x+w].shape
        pred = cv2.resize(pred, (s[1], s[0]), interpolation=cv2.INTER_NEAREST)
        pred_mask[y:y+h, x:x+w] = pred * (counter)
        pred = torch.tensor(pred, dtype=torch.int32)
        pred_masks_list.append(pred)
      except Exception as e:
        # print(str(e))
        break
    gt_mask = torch.tensor(gt_mask, dtype=torch.int32)
    pred_mask = torch.tensor(pred_mask, dtype=torch.int32)
    return img, gt_mask, pred_mask, gt_masks_list, pred_masks_list # gt_mask could be all zero when the ground truth is not given.


### Visualization and Submission

In [ ]:
'''
# Visualise the output prediction as well as the GT Mask and Input image for a sample input
# TODO: approx 10 lines
'''

test_dataset_dicts = get_detection_data("train")
for d in random.sample(test_dataset_dicts, 1):  
  sample = d
  sample['image_id'] = sample['file_name']
  img, true_mask, pred_mask, gt_masks_list, pred_masks_list = get_prediction_mask(sample)

  import matplotlib.pyplot as plt
  plt.figure(figsize=(15,10))
  plt.imshow(pred_mask, cmap='gray')
  plt.figure(figsize=(15,10))
  plt.imshow(true_mask, cmap='gray')
  plt.figure(figsize=(15,10))
  plt.imshow(img)

In [ ]:
'''
# Visualise the output prediction and Input image for a test input
# TODO: approx 10 lines
'''

test_dataset_dicts = get_detection_data("test")
for d in random.sample(test_dataset_dicts, 1):  
  sample = d
  sample['image_id'] = sample['file_name']
  img, true_mask, pred_mask, gt_masks_list, pred_masks_list = get_prediction_mask(sample)

  import matplotlib.pyplot as plt
  plt.figure(figsize=(15,10))
  plt.imshow(pred_mask, cmap='gray')
  plt.figure(figsize=(15,10))
  plt.imshow(img)

In [ ]:
'''
# ref: https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
# https://www.kaggle.com/c/airbus-ship-detection/overview/evaluation
'''
def rle_encoding(x):
    '''
    x: pytorch tensor on gpu, 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = torch.where(torch.flatten(x.long())==1)[0]
    if(len(dots)==0):
      return []
    inds = torch.where(dots[1:]!=dots[:-1]+1)[0]+1
    inds = inds.cuda()
    inds = torch.cat((torch.tensor([0], device=torch.device('cuda'), dtype=torch.long), inds))
    tmpdots = dots[inds]
    tmpdots = tmpdots.cuda()
    inds = torch.cat((inds, torch.tensor([len(dots)], device=torch.device('cuda'))))
    inds = inds[1:] - inds[:-1]
    runs = torch.cat((tmpdots, inds)).reshape((2,-1))
    runs = torch.flatten(torch.transpose(runs, 0, 1)).cpu().data.numpy()
    return ' '.join([str(i) for i in runs])

In [ ]:
'''
# You need to upload the csv file on kaggle
# The speed of your code in the previous parts highly affects the running time of this part
'''

preddic = {"ImageId": [], "EncodedPixels": []}

'''
# Writing the predictions of the training set
'''
my_data_list = DatasetCatalog.get("data_detection_{}".format('train'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name']
  img, true_mask, pred_mask, gt_masks_list, pred_masks_list = get_prediction_mask(sample)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index)
      encPix = rle_encoding(tmp_mask)
      sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

print(len(preddic["ImageId"]))

'''
# Writing the predictions of the test set
'''

my_data_list = DatasetCatalog.get("data_detection_{}".format('test'))
for i in tqdm(range(len(my_data_list)), position=0, leave=True):
  sample = my_data_list[i]
  sample['image_id'] = sample['file_name']
  img, true_mask, pred_mask, gt_masks_list, pred_masks_list = get_prediction_mask(sample)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index)
      encPix = rle_encoding(tmp_mask)
      sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

print(len(preddic["ImageId"]))

pred_file = open("{}/pred.csv".format(BASE_DIR), 'w')
pd.DataFrame(preddic).to_csv(pred_file, index=False)
pred_file.close()


## Part 4: Mask R-CNN

For this part you need to follow a same procedure to part 2 with the configs of Mask R-CNN, other parts are generally the same as part 2.

### Data Loader

In [ ]:
dataset_dicts_train = get_detection_data("train")
dataset_dicts_test = get_detection_data("test")

### Network

In [ ]:
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_detection_train",)
cfg.DATASETS.TEST = ("data_detection_test",)
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.003 
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
cfg.SOLVER.STEPS = []
cfg.MODEL.PIXEL_STD = [57.375, 57.120, 58.395]
cfg.SOLVER.MOMENTUM = 0.9
cfg.SOLVER.MAX_ITER = 2000  
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512  
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 

### Training

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer_4 = DefaultTrainer(cfg) 
trainer_4.resume_or_load(resume=False)
trainer_4.train()

### Evaluation and Visualization

In [ ]:

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6 
predictor_4 = DefaultPredictor(cfg)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("data_detection_train", output_dir="/content/drive/MyDrive/CMPT_CV_lab3/data/output/")
test_loader = build_detection_test_loader(cfg, "data_detection_train")
print(inference_on_dataset(predictor_4.model, test_loader, evaluator))

In [ ]:
from detectron2.data.build import get_detection_dataset_dicts
from detectron2.utils.visualizer import ColorMode
test_dataset_dicts = get_detection_data("test")
for d in random.sample(test_dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    # im = torch.tensor(im, device=torch.device('cuda'), dtype=torch.float)
    outputs = predictor_4(im)  
    v = Visualizer(im[:, :, ::-1],
                    metadata=data_detection_test_metadata, 
                    scale=0.5, 
                    instance_mode=ColorMode.IMAGE_BW 
      )
    out = v.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])